In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# ---Content Based Filterting Process

##### * 컨텐츠에 대한 feature를 vectorize
##### * feature들 간의 cosine similarity를 사용하여 유사도 행렬 구한다
##### * 부가적으로 컨텐츠별 평점수와 평점을 사용하여 가중평점 계산
###### ( 이유: 적은 평점 수로 인한 평균평점 상향되는 현상 고려)
##### * 컨텐츠의 유사도와 가중평점이 높은 순으로 추천

In [3]:
path = '/content/drive/Shareddrives/DSL_2021_2_추천시스템/animedataset/'

In [4]:
anime = pd.read_csv(path + 'sample_top_100000.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (6,30,31,35,36,37,38) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
anime = anime[:10000]

- computing complexity too high for cosine similarity 

In [6]:
anime.columns

Index(['user_id', 'anime_id', 'rating', 'watching_status', 'watched_episodes',
       'Name', 'Score', 'Genres', 'English name', 'Japanese name', 'Type',
       'Episodes', 'Aired', 'Premiered', 'Producers', 'Licensors', 'Studios',
       'Source', 'Duration', 'Rating', 'Ranked', 'Popularity', 'Members',
       'Favorites', 'Watching', 'Completed', 'On-Hold', 'Dropped',
       'Plan to Watch', 'Score-10', 'Score-9', 'Score-8', 'Score-7', 'Score-6',
       'Score-5', 'Score-4', 'Score-3', 'Score-2', 'Score-1'],
      dtype='object')

In [7]:
anime = anime[['anime_id','Score','Name','Genres','Score-10', 'Score-9', 'Score-8', 'Score-7', 'Score-6',
       'Score-5', 'Score-4', 'Score-3', 'Score-2', 'Score-1']]

In [8]:
#만화에 평점을 준 사람 총합
anime['vote_count'] = anime.iloc[:,4:].sum(axis=1)

anime.drop(['Score-10', 'Score-9', 'Score-8', 'Score-7', 'Score-6',
       'Score-5', 'Score-4', 'Score-3', 'Score-2', 'Score-1'], axis=1, inplace=True)

In [9]:
anime.Score.replace('Unknown',np.nan, inplace=True)
anime = anime.dropna()

In [10]:
anime.Score = anime.Score.astype(np.float64)

In [11]:
anime = anime.reset_index(drop=True)

In [12]:
anime['Genres'] = [x.replace(',','') for x in anime.Genres]

# Case1 :  가중평점 고려 X

In [13]:
count = TfidfVectorizer(ngram_range=(1,3))

In [14]:
count_vector_genres = count.fit_transform(anime['Genres'])

In [16]:
count_vector_genres.shape

(9992, 2590)

In [17]:
genre_sim = cosine_similarity(count_vector_genres,count_vector_genres)
print(genre_sim)

[[1.         0.01742386 0.06031213 ... 0.00708682 0.         0.00843677]
 [0.01742386 1.         0.04900962 ... 0.0539762  0.05776216 0.11029674]
 [0.06031213 0.04900962 1.         ... 0.03192938 0.01988292 0.0080372 ]
 ...
 [0.00708682 0.0539762  0.03192938 ... 1.         0.69796914 0.0400686 ]
 [0.         0.05776216 0.01988292 ... 0.69796914 1.         0.08250722]
 [0.00843677 0.11029674 0.0080372  ... 0.0400686  0.08250722 1.        ]]


In [18]:
genre_sim_idx = genre_sim.argsort()[::-1]

In [19]:
genre_sim_idx

array([[4995, 2477, 2478, ..., 1279, 9963, 9991],
       [   0, 4760, 4759, ..., 5400, 9989, 9990],
       [4995, 4980, 1465, ..., 6369, 9990, 9989],
       ...,
       [4931, 5190, 5188, ..., 6375, 4053, 4413],
       [4995, 5535, 5530, ..., 7427,  471, 6850],
       [3371, 7779, 1533, ..., 6848, 4935,    0]])

In [20]:
def recommend_MANHWA(df, sorted_idx, title_name, top_n=10):

    anime_title_idx = df[df['Name'] == title_name].index.values
    sim_idx = genre_sim_idx[anime_title_idx, :top_n].reshape(-1,)
    similar_anime = df.iloc[sim_idx].sort_values('Score',ascending=False)[:10]
  

    return similar_anime

In [21]:
recommend_MANHWA(anime, genre_sim_idx, 'Keroro Gunsou')


,anime_id,Score,Name,Genres,vote_count
3917,13759,8.19,Sakura-sou no Pet na Kanojo,Slice of Life Comedy Drama Romance School,231135.0
7457,14813,8.07,Yahari Ore no Seishun Love Comedy wa Machigatt...,Slice of Life Comedy Drama Romance School,239027.0
3078,16049,8.06,Toaru Kagaku no Railgun S,Action Sci-Fi Super Power,63181.0
7453,35507,7.85,Youkoso Jitsuryoku Shijou Shugi no Kyoushitsu ...,Slice of Life Psychological Drama School,161627.0
6024,16904,7.70,K: Missing Kings,Action Super Power Supernatural,51026.0
3922,33988,7.62,Demi-chan wa Kataritai,Comedy Vampire Fantasy School Seinen,83608.0
6028,14467,7.51,K,Action Mystery Super Power Supernatural,167587.0
7454,24627,7.46,Yamada-kun to 7-nin no Majo: Mou Hitotsu no Su...,Comedy Romance School Shounen,22444.0
9991,19251,7.27,Uta no☆Prince-sama♪ Maji Love 2000%: Shining S...,Comedy Harem Music Romance School Shoujo,5891.0
3921,147,7.24,Kimi ga Nozomu Eien,Drama Romance Slice of Life,33555.0


In [22]:
anime[anime.Name=='Keroro Gunsou']

,anime_id,Score,Name,Genres,vote_count
1549,516,7.7,Keroro Gunsou,Comedy Mecha Parody Sci-Fi Shounen,15008.0
5486,516,7.7,Keroro Gunsou,Comedy Mecha Parody Sci-Fi Shounen,15008.0


# Case2 : 가중평균 고려 O

In [23]:
C = anime['Score'].mean()

anime['vote_count'] = anime['vote_count'].astype(int)
m = anime['vote_count'].quantile(0.9)



In [24]:
def weighted_rating (x, m=m,C=C):
    V = x['vote_count']
    R = x['Score']
    
    return (V/(V+m) * R ) + (m/(m+V) * C)

In [25]:
WA_anime = anime.copy()
WA_anime['Score'] = WA_anime.apply(weighted_rating, axis=1)

In [26]:
recommend_MANHWA(anime, genre_sim_idx, 'Keroro Gunsou')

,anime_id,Score,Name,Genres,vote_count
3917,13759,8.19,Sakura-sou no Pet na Kanojo,Slice of Life Comedy Drama Romance School,231135
7457,14813,8.07,Yahari Ore no Seishun Love Comedy wa Machigatt...,Slice of Life Comedy Drama Romance School,239027
3078,16049,8.06,Toaru Kagaku no Railgun S,Action Sci-Fi Super Power,63181
7453,35507,7.85,Youkoso Jitsuryoku Shijou Shugi no Kyoushitsu ...,Slice of Life Psychological Drama School,161627
6024,16904,7.70,K: Missing Kings,Action Super Power Supernatural,51026
3922,33988,7.62,Demi-chan wa Kataritai,Comedy Vampire Fantasy School Seinen,83608
6028,14467,7.51,K,Action Mystery Super Power Supernatural,167587
7454,24627,7.46,Yamada-kun to 7-nin no Majo: Mou Hitotsu no Su...,Comedy Romance School Shounen,22444
9991,19251,7.27,Uta no☆Prince-sama♪ Maji Love 2000%: Shining S...,Comedy Harem Music Romance School Shoujo,5891
3921,147,7.24,Kimi ga Nozomu Eien,Drama Romance Slice of Life,33555
